In [4]:
pip install lyricsgenius

Note: you may need to restart the kernel to use updated packages.


In [19]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import os
import re
import pandas as pd
import json
from bs4 import BeautifulSoup
import azapi

In [6]:
## CLIENT ID AND CLIENT SECRET
### DO NOT PUT ON GIT

client_id = ''
client_secret = ''
genius_access_token = ''


In [7]:
def get_top_tracks(artist_name, client_id, client_secret):
    # Initialize Spotify client with your credentials
    credentials = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=credentials)
    
    # search for the artist to get the Spotify ID through sp.search
    results = sp.search(q='artist:' + artist_name, type='artist') 
    items = results['artists']['items']
    if not items:
        return []
    
    
    artist_id = items[0]['id']
    
    top_tracks = sp.artist_top_tracks(artist_id)
    
    return [track['name'] for track in top_tracks['tracks']]

In [8]:
artist_name = 'Selena Gomez'
top_tracks = get_top_tracks(artist_name, client_id, client_secret)
top_tracks

['Calm Down (with Selena Gomez)',
 'Lose You To Love Me',
 'Single Soon',
 'It Ain’t Me (with Selena Gomez)',
 'Love On',
 'People You Know',
 'Wolves',
 'Back To You - From 13 Reasons Why – Season 2 Soundtrack',
 'Fetish (feat. Gucci Mane)',
 'Same Old Love']

In [9]:
""" 
def get_lyrics(song_name, artist_name, genius_access_token):
    #set up the request using access token 
    base_url = "https://api.genius.com"
    headers = {'Authorization': 'Bearer ' + genius_access_token}
    search_url = base_url + "/search" #getting ready to search!
    data = {'q': song_name} #we want to search the song name!

    response = requests.get(search_url, params=data, headers=headers)
    json = response.json()
    path = None

    # loops through the search results returned by the Genius API to find a specific song that matches name of song and artist name

    for hit in json["response"]["hits"]:
        if artist_name.lower() in hit["result"]["primary_artist"]["name"].lower():
            path = hit["result"]["path"]
            break #assuming the first is right 

    if path:
        lyrics_url = "https://genius.com" + path
        lyrics_response = requests.get(lyrics_url)
        soup = BeautifulSoup(lyrics_response.text, 'lxml')
        print(soup)

        # Check if the song is instrumental
        instrumental_div = soup.find('div', class_='LyricsPlaceholder__Message-uen8er-2')
        if instrumental_div:
            return "This song is an instrumental"

        lyrics_div = soup.find('div', {'data-lyrics-container': 'true'})
        lyrics = lyrics_div.get_text(separator='\n', strip=True) if lyrics_div else "Lyrics not found"
        return lyrics

    else:
        return "Lyrics not found" 
"""

' \ndef get_lyrics(song_name, artist_name, genius_access_token):\n    #set up the request using access token \n    base_url = "https://api.genius.com"\n    headers = {\'Authorization\': \'Bearer \' + genius_access_token}\n    search_url = base_url + "/search" #getting ready to search!\n    data = {\'q\': song_name} #we want to search the song name!\n\n    response = requests.get(search_url, params=data, headers=headers)\n    json = response.json()\n    path = None\n\n    # loops through the search results returned by the Genius API to find a specific song that matches name of song and artist name\n\n    for hit in json["response"]["hits"]:\n        if artist_name.lower() in hit["result"]["primary_artist"]["name"].lower():\n            path = hit["result"]["path"]\n            break #assuming the first is right \n\n    if path:\n        lyrics_url = "https://genius.com" + path\n        lyrics_response = requests.get(lyrics_url)\n        soup = BeautifulSoup(lyrics_response.text, \'lxml\

In [10]:
"""
API = azapi.AZlyrics(accuracy=0.5)
def get_lyrics(song, artist):
    #using azapi https://github.com/elmoiv/azapi 
    API.artist = artist
    API.title = song
    lyrics = API.getLyrics(save=False, ext='lrc')
    #print(API.title, API.artist)
    if lyrics == 0:
        return "Lyrics not found"
    else:
        return lyrics
"""

'\nAPI = azapi.AZlyrics(accuracy=0.5)\ndef get_lyrics(song, artist):\n    #using azapi https://github.com/elmoiv/azapi \n    API.artist = artist\n    API.title = song\n    lyrics = API.getLyrics(save=False, ext=\'lrc\')\n    #print(API.title, API.artist)\n    if lyrics == 0:\n        return "Lyrics not found"\n    else:\n        return lyrics\n'

In [11]:
from lyricsgenius import Genius

genius = Genius(genius_access_token, sleep_time=0)
#genius = Genius(access_token = genius_access_token, remove_section_headers = True, excluded_terms = ["(Remix)", "(Live)", "(Clean)", "(Explicit)", "Slowed", "Sped Up", "Instrumental", "instrumental"])

In [12]:
from requests.exceptions import Timeout

def get_lyrics(song, artist):
    #using lyricsgenius 
    genius.timeout = 15
    genius.sleep_time = 5  
    track = genius.search_song(song, artist)
    retries = 0
    while retries < 3:
        try:
            lyrics = re.sub(r'\[.*?\]', '', track.lyrics)
            lyrics = re.sub(r'.*Lyrics\n', '', lyrics)
            clean_lyrics = re.sub(r'\d+Embed$', '', lyrics)
            return clean_lyrics
        except Timeout as e:
                retries += 1
                continue
        except:
            print("Lyrics not found")
        break
    #return track

In [13]:
lyrics = get_lyrics('SOLO', 'JENNIE')
print(lyrics)

Searching for "SOLO" by JENNIE...
Done.


천진난만 청순가련
새침한 척 이젠 지쳐 나
귀찮아
매일 뭐 해 어디야 밥은 잘 자
Baby, 자기 여보 보고 싶어
다 부질없어

You got me like 
(Oh-oh-oh) 이건 아무 감동 없는 love story 
(Oh-oh-oh) 어떤 설렘도 어떤 의미도 
(Oh-oh-oh) 네겐 미안하지만 I'm not sorry 
(Oh-oh-oh) 오늘부터 난 난 난

빛이 나는 솔로
빛이 나는 솔로
I'm goin' solo-lo-lo-lo-lo-lo-lo
I'm goin' solo-lo-lo-lo-lo-lo-lo


Used to be your girl
Now I'm used to being the GOAT
You're sittin' on your feelings
I'm sittin' on my throne
I ain't got no time for the troubles in your eyes
This time I'm only lookin' at me, myself and I
(I'm goin' solo) I'ma do it on my own now
Now that you're alone, got you lookin' for a clone now
(So low) That's how I'm gettin' down
Destined for this and the crown
You might also like
Sing it loud like 
(Oh-oh-oh) 이건 아무 감동 없는 love story 
(Oh-oh-oh) 어떤 설렘도 어떤 의미도
(Oh-oh-oh, ooh) 네겐 미안하지만 I'm not sorry 
(Oh-oh-oh) 오늘부터 난 난 난


빛이 나는 솔로
빛이 나는 솔로
I'm goin' solo-lo-lo-lo-lo-lo-lo
I'm goin' solo-lo-lo-lo-lo-lo-lo


만남 설렘 감동 뒤엔
이별 눈물 후회 그리움
홀로인 게 좋아 난 나다워야 하니

In [14]:
cwd = os.getcwd()
cwd
viral_songs = f'{cwd}/viral.csv'
viral_songs = pd.read_csv(viral_songs)
viral_songs

,artists,songs
0,Kinfolk Thugs,Dumptruck
1,Cordelia,Little Life
2,"¥$, Kanye West, Ty Dolla $ign & Rich The Kid",CARNIVAL (feat. Playboi Carti)
3,Ariana Grande,"yes, and? (Mixed)"
4,Bobby Caldwell,What You Won't Do for Love
5,R.m.y,How I Love Being a Woman 6
6,Rihanna,Bitch Better Have My Money
7,"The Weeknd, JENNIE & Lily Rose Depp",One Of The Girls
8,Gold-Tiger,Funny
9,Beyoncé,Countdown


In [15]:
from collections import Counter

singers = []
artists = Counter(viral_songs['artists'])
for artist in artists: 
    singers.append(artist)

singers

['Kinfolk Thugs',
 'Cordelia',
 '¥$, Kanye West, Ty Dolla $ign & Rich The Kid',
 'Ariana Grande',
 'Bobby Caldwell',
 'R.m.y',
 'Rihanna',
 'The Weeknd, JENNIE & Lily Rose Depp',
 'Gold-Tiger',
 'Beyoncé',
 'Masego & FKJ',
 'Lana Del Rey',
 'Peso Pluma & Anitta',
 '21 Savage',
 'Flo Milli',
 'Muni Long',
 'ผดุง ทรงแสง (แจ๊ส)',
 'Lilithzplug']

In [16]:
for singer in singers: 
    full_artists = [x.strip() for artist in artists for x in re.split(r',|&', artist)]
full_artists

['Kinfolk Thugs',
 'Cordelia',
 '¥$',
 'Kanye West',
 'Ty Dolla $ign',
 'Rich The Kid',
 'Ariana Grande',
 'Bobby Caldwell',
 'R.m.y',
 'Rihanna',
 'The Weeknd',
 'JENNIE',
 'Lily Rose Depp',
 'Gold-Tiger',
 'Beyoncé',
 'Masego',
 'FKJ',
 'Lana Del Rey',
 'Peso Pluma',
 'Anitta',
 '21 Savage',
 'Flo Milli',
 'Muni Long',
 'ผดุง ทรงแสง (แจ๊ส)',
 'Lilithzplug']

In [17]:
for artist in full_artists: 
    tracks = get_top_tracks(artist, client_id, client_secret)
    print(artist)
    for track in tracks:
        print(track)
    print()

Kinfolk Thugs
Back It up and Dump It (Dump Truck)
Dumptruck
Dump Truck
Dump Truck Remix (Clean)
I Speak Memphis
Who Gon Check It
Dump Truck Remix (Explicit)
Back Seat (Pull It Over)
Money Maker
Dump Truck (Back it up and Dump it) - instrumental

Cordelia
Little Life
I Just Killed a Spider
Caramel
Play Pretend
What I Wouldn't Do
Little Life - radio edit cos they made me
Not So Sure

¥$
CARNIVAL
BURN
FUK SUMN
BACK TO ME
DO IT
STARS
TALKING
PAID
PAPERWORK
VULTURES

Kanye West
CARNIVAL
Heartless
Flashing Lights
Father Stretch My Hands Pt. 1
BURN
Runaway
FUK SUMN
Ni**as In Paris
I Wonder
Bound 2

Ty Dolla $ign
CARNIVAL
BURN
FUK SUMN
Or Nah (feat. The Weeknd, Wiz Khalifa & DJ Mustard) - Remix
BACK TO ME
DO IT
STARS
Nights Like This (feat. Ty Dolla $ign)
WUSYANAME (feat. Youngboy Never Broke Again & Ty Dolla $ign)
TALKING

Rich The Kid
CARNIVAL
Plug Walk
New Freezer (feat. Kendrick Lamar)
Splashin
TAlk tO Me (with Rich The Kid)
Real One (feat. Rich The Kid)
You & Jennifer (the other side)
Pes

In [20]:
lyrics = {}
for artist in full_artists:
    lyrics[artist] = {'songs': []}
    tracks = get_top_tracks(artist, client_id, client_secret)
    for track in tracks:
        song_lyrics = get_lyrics(track, artist)
        lyrics[artist]['songs'].append({
            'song_name': track,
            'lyrics': song_lyrics
        })

output_file = "top_lyrics.json"

with open(output_file, "w") as json_file:
    json.dump(lyrics, json_file)

print("Lyrics saved to", output_file)

Searching for "Back It up and Dump It (Dump Truck)" by Kinfolk Thugs...
Done.
Searching for "Dumptruck" by Kinfolk Thugs...
Done.
Searching for "Dump Truck" by Kinfolk Thugs...
Done.
Searching for "Dump Truck Remix (Clean)" by Kinfolk Thugs...
Done.
Searching for "I Speak Memphis" by Kinfolk Thugs...
Done.
Searching for "Who Gon Check It" by Kinfolk Thugs...
Done.
Searching for "Dump Truck Remix (Explicit)" by Kinfolk Thugs...
No results found for: 'Dump Truck Remix (Explicit) Kinfolk Thugs'
Lyrics not found
Searching for "Back Seat (Pull It Over)" by Kinfolk Thugs...
Done.
Searching for "Money Maker" by Kinfolk Thugs...
Done.
Searching for "Dump Truck (Back it up and Dump it) - instrumental" by Kinfolk Thugs...
Done.
Searching for "Little Life" by Cordelia...
Done.
Searching for "I Just Killed a Spider" by Cordelia...
Specified song does not contain lyrics. Rejecting.
Lyrics not found
Searching for "Caramel" by Cordelia...
Done.
Searching for "Play Pretend" by Cordelia...
Done.
Search